In [1]:
import os
import random
import pandas as pd
import numpy as np
import warnings
from typing import Optional
from tqdm import tqdm as tq
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, MarianMTModel, MarianTokenizer
from transformers import logging
import sentencepiece
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="2,3"
warnings.filterwarnings(action='ignore')
logging.set_verbosity_error()

# 임시 토크나이저
from konlpy.tag import Okt, Komoran

# Configuration
CFG = {
    'SEED':1203,
    'EPOCHS':10,
    'LEARNING_RATE':0.001,
    'BATCH_SIZE':256,
    'PLM':"klue/roberta-large",
    'MAX_LEN':48,
}

In [2]:
# load data
folder = os.getcwd() + '/open'
train = "train.csv"
test  = "test.csv"
submit = "sample_submission.csv"
train = pd.read_csv(folder + '/' + train)

# augmentation applied data
aug1 = pd.read_csv("대화형_aug.csv", index_col = 0)
aug2 = pd.read_csv("예측형_aug.csv", index_col = 0)
aug3 = pd.read_csv("추론형_aug.csv", index_col = 0)
train = pd.concat([train, aug1, aug2, aug3], 0)
del train['ID']
train

,문장,유형,극성,시제,확실성,label
0,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실
3,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
...,...,...,...,...,...,...
TRAIN_07796,우리가 할 수 있는 일이 없다는 것을 알아야 합니다,추론형,긍정,미래,확실,추론형-긍정-미래-확실
TRAIN_14982,당신은 당신이 계약을 어기지 않았다고 주장할 수 있습니다,추론형,긍정,미래,불확실,추론형-긍정-미래-불확실
TRAIN_00048,원칙과 기준을 아는 것이 큰 도움이 될 것입니다,추론형,긍정,미래,확실,추론형-긍정-미래-확실
TRAIN_01233,산업과 지역의 경제를 활성화하는 데 사용할 수 있습니다,추론형,긍정,현재,확실,추론형-긍정-현재-확실


In [3]:
os.listdir('open')

['train.csv', 'test.csv', 'sample_submission.csv']

In [4]:
# load data
folder = os.getcwd() + '/open'
train = 'train.csv'
test  = 'test.csv'
submit = 'sample_submission.csv'
train = pd.read_csv(folder + '/' + train)
train, valid = train_test_split(train, test_size=0.2, random_state=CFG['SEED'])

test = pd.read_csv(folder + '/' + test)
submit = pd.read_csv(folder + '/' + submit)
print(f"1. num train : {len(train)}\n")
print(f"2. null train check : \n{train.isnull().sum()}\n")
print(f"3. unique labels : \n{train['유형'].unique()}\n\
{[len(train[train['유형']==train['유형'].unique()[idx]]) for idx in range(len(train['유형'].unique()))]}\n\
{train['극성'].unique()}\n\
{[len(train[train['극성']==train['극성'].unique()[idx]]) for idx in range(len(train['극성'].unique()))]}\n\
{train['시제'].unique()}\n\
{[len(train[train['시제']==train['시제'].unique()[idx]]) for idx in range(len(train['시제'].unique()))]}\n\
{train['확실성'].unique()}\n\
{[len(train[train['확실성']==train['확실성'].unique()[idx]]) for idx in range(len(train['확실성'].unique()))]}\n"
)

1. num train : 13232

2. null train check : 
ID       0
문장       0
유형       0
극성       0
시제       0
확실성      0
label    0
dtype: int64

3. unique labels : 
['추론형' '사실형' '대화형' '예측형']
[1709, 10852, 463, 208]
['긍정' '부정' '미정']
[12627, 464, 141]
['현재' '미래' '과거']
[5493, 1339, 6400]
['확실' '불확실']
[12142, 1090]



In [ ]:
# # preprocess data
# def truncate(texts, data):
#     truncated_texts = []
#     for txt in texts:
#         splited = txt.split(' ')
#         if len(splited)>=64:
#             valid = ' '.join(splited[:32] + splited[-32:])
#             truncated_texts.append(valid)
#         else:
#             truncated_texts.append(txt)  
#     data['문장'] = truncated_texts
#     return

# truncate(train['문장'], train)
# truncate(valid['문장'], valid)
# truncate(test ['문장'], test)

# train = train.reset_index()
# valid = valid.reset_index()
# test = test.reset_index()

In [5]:
embeddings = AutoModel.from_pretrained(CFG["PLM"]).embeddings

Downloading:   0%|          | 0.00/547 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(CFG['PLM'])    
text = train['문장'][30]

encoding = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=128,
    return_token_type_ids=False,
    padding = 'max_length',
    truncation = True,
    return_attention_mask=True,
    return_tensors='pt'
)

x_ids, x_attn = [encoding['input_ids'], encoding['attention_mask']]

In [14]:
embeddings(x_ids).shape

torch.Size([1, 128, 1024])

In [5]:
# preprocess data
def type_truncate(texts, data):   # 8개 어절만 본다!
    truncated_texts = []
    for txt in texts:
        splited = txt.split(' ')
        if len(splited)>=8:
            valid = ' '.join(splited[-8:])
            truncated_texts.append(valid)
        else:
            truncated_texts.append(txt)  
    data['문장'] = truncated_texts
    return data

type_train = type_truncate(train['문장'], train)
type_valid = type_truncate(valid['문장'], valid)
type_test  = type_truncate(test ['문장'], test)

type_train = type_train.reset_index()
type_valid = type_valid.reset_index()
type_test  = type_test.reset_index()

In [6]:
type_train.groupby('유형').count()['ID']

유형
대화형      463
사실형    10852
예측형      208
추론형     1709
Name: ID, dtype: int64

In [7]:
type_train.groupby('극성').count()['ID']

극성
긍정    12627
미정      141
부정      464
Name: ID, dtype: int64

In [8]:
type_train.groupby('시제').count()['ID']

시제
과거    6400
미래    1339
현재    5493
Name: ID, dtype: int64

In [9]:
type_train.groupby('확실성').count()['ID']

확실성
불확실     1090
확실     12142
Name: ID, dtype: int64

# MT 기반 augmentation

In [12]:
from transformers import pipeline
ko_to_en = pipeline("text2text-generation", model = "circulus/kobart-trans-ko-en-v2")
en_to_ko = pipeline("text2text-generation", model = "circulus/kobart-trans-en-ko-v2")




### 예측형

In [15]:
import copy
from tqdm import tqdm as tq

예측형= type_train[type_train['유형']=='예측형']
예측형_aug = copy.deepcopy(예측형)
예측형_augtext = []

for text in tq(예측형.문장):
    translated = ko_to_en(text, max_length = 1024)[0]['generated_text']
    translated = en_to_ko(translated, max_length = 1024)[0]['generated_text']
    예측형_augtext.append(translated)
    
예측형_aug['문장'] = 예측형_augtext

invalid_index = []
for idx in range(len(예측형_augtext)):
    if 예측형_augtext[idx][-1] =='다':
        continue
    invalid_index.append(idx)

예측형_aug = 예측형_aug.reset_index()
del 예측형_aug['level_0'], 예측형_aug["index"]
예측형_aug = 예측형_aug.drop(index=invalid_index).reset_index()
del 예측형_aug['index']
예측형_aug

100%|████████████████████████████████████████████████████████| 208/208 [03:16<00:00,  1.06it/s]


,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_09117,NFT를 판매할 예정이며 게임별로 최적화된 토큰 발행도 예측하고 있습니다,예측형,긍정,과거,확실,예측형-긍정-과거-확실
1,TRAIN_13768,그가 등장하면 좌완 김광현에게 더 값진 존재가 될 것으로 보인다,예측형,긍정,미래,불확실,예측형-긍정-미래-불확실
2,TRAIN_05598,동해와 서해 남해에서 약 200km 이내 먼바다의 파고는 05∼1m로 예상된다,예측형,긍정,현재,불확실,예측형-긍정-현재-불확실
3,TRAIN_03300,빚이 났지만 앞으로 자동차 산업 전반에 피해가 확산될 것으로 예상된다,예측형,긍정,미래,불확실,예측형-긍정-미래-불확실
4,TRAIN_00797,참여 은공업 발전 방향에 대한 포럼도 열린다,예측형,긍정,미래,불확실,예측형-긍정-미래-불확실
...,...,...,...,...,...,...,...
201,TRAIN_09237,성장하면서 맑고 화창한 날씨가 이어질 것으로 예상됩니다,예측형,긍정,미래,불확실,예측형-긍정-미래-불확실
202,TRAIN_11118,규모로 선정하여 교육과정을 보안기술단위로 세분화하여 운영할 예정입니다,예측형,긍정,미래,불확실,예측형-긍정-미래-불확실
203,TRAIN_01628,주요 지분을 만들어 보유하는 방식을 택할 것으로 보인다,예측형,긍정,미래,불확실,예측형-긍정-미래-불확실
204,TRAIN_10263,예전보다 강하다는 점에서 정면승부가 예상된다,예측형,긍정,미래,불확실,예측형-긍정-미래-불확실


In [21]:
예측형_aug.to_csv("예측형_aug.csv", index=False)

### 대화형 -> 니다, 요, 까, 나, 군, 야, 여, 냐, 죠

In [ ]:
import copy
from tqdm import tqdm as tq

대화형= train[train['유형']=='대화형']
대화형_aug = copy.deepcopy(대화형)
대화형_augtext = []

for text in tq(대화형.문장):
    translated = ko_to_en(text, max_length = 1024)[0]['generated_text']
    translated = en_to_ko(translated, max_length = 1024)[0]['generated_text']
    대화형_augtext.append(translated)
    
대화형_aug['문장'] = 대화형_augtext
invalid_index = []
end_word_2 = set(["니다", "시다"])
end_word_1 = set(["요", "까", "나", "군", "야", "여", "냐", "죠", "어", "마", "해", "자", "봐", "여보"])

for idx in range(len(대화형_augtext)):
    if 대화형_augtext[idx][-2:] in end_word_2:
        continue
    elif 대화형_augtext[idx][-1] in end_word_1:
        continue
    invalid_index.append(idx)

대화형_aug = 대화형_aug.reset_index()
del 대화형_aug["index"]
대화형_aug = 대화형_aug.drop(index=invalid_index).reset_index()
del 대화형_aug['index']



In [19]:
대화형_aug

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_06877,우선 흙의 대명사는 정말 오해입니다,대화형,긍정,현재,확실,대화형-긍정-현재-확실
1,TRAIN_13914,연습 방법도 있어요,대화형,긍정,현재,확실,대화형-긍정-현재-확실
2,TRAIN_05399,왜 1차 검사에서 음성 판정이 나오고 재검사에서 양성 판정이 나오나요,대화형,긍정,현재,불확실,대화형-긍정-현재-불확실
3,TRAIN_07492,가속을 멈추세요,대화형,긍정,현재,확실,대화형-긍정-현재-확실
4,TRAIN_02264,확장 등 시장에 필요한 정책이 소홀한 것 아닌가요,대화형,긍정,현재,불확실,대화형-긍정-현재-불확실
...,...,...,...,...,...,...,...
271,TRAIN_02519,먼저 거울을 이용하여 주기적으로 물의 뒤쪽 뒤쪽과 머리의 안쪽을 살펴보세요,대화형,긍정,현재,확실,대화형-긍정-현재-확실
272,TRAIN_04705,볶음밥과 볶음밥이 비슷할 수 있지 않을까요,대화형,부정,현재,불확실,대화형-부정-현재-불확실
273,TRAIN_05143,바위에 누워서 일광욕을 할 수 있다면 얼마나 좋을까요,대화형,긍정,현재,불확실,대화형-긍정-현재-불확실
274,TRAIN_14935,추운 암흑에도 도움이 되고 산소가 없으면 완벽합니다,대화형,긍정,현재,확실,대화형-긍정-현재-확실


In [20]:
대화형_aug.to_csv("대화형_aug.csv", index=False)

### 추론형

In [22]:
import copy
from tqdm import tqdm as tq

추론형= train[train['유형']=='추론형']
추론형_aug = copy.deepcopy(추론형)
추론형_augtext = []

for text in tq(추론형.문장):
    translated = ko_to_en(text, max_length = 1024)[0]['generated_text']
    translated = en_to_ko(translated, max_length = 1024)[0]['generated_text']
    추론형_augtext.append(translated)
    
추론형_aug['문장'] = 추론형_augtext



100%|██████████████████████████████████████████████████████| 1709/1709 [22:51<00:00,  1.25it/s]


In [24]:
추론형_aug

,ID,문장,유형,극성,시제,확실성,label
11896,TRAIN_11896,편집자들은 최근 누군가가 공유하는 동영상 시청에 반했다,추론형,긍정,현재,확실,추론형-긍정-현재-확실
6880,TRAIN_06880,그럼,추론형,긍정,현재,확실,추론형-긍정-현재-확실
6661,TRAIN_06661,인센티브를 사용하여 뛰어들기도 합니다,추론형,긍정,미래,불확실,추론형-긍정-미래-불확실
12997,TRAIN_12997,다양한 관행과 규정도 어렵습니다,추론형,긍정,현재,확실,추론형-긍정-현재-확실
3559,TRAIN_03559,카카오게임즈는 모바일 게임 시장 매출 톱3 진입을 자신하고 있다,추론형,긍정,현재,확실,추론형-긍정-현재-확실
...,...,...,...,...,...,...,...
11827,TRAIN_11827,마스크를 써야 해요,추론형,긍정,현재,확실,추론형-긍정-현재-확실
11569,TRAIN_11569,조건도 더 좋아요,추론형,긍정,현재,확실,추론형-긍정-현재-확실
16031,TRAIN_16031,그 이유는 제가 안전자산에 지나치게 집착하기 때문입니다,추론형,긍정,현재,확실,추론형-긍정-현재-확실
4136,TRAIN_04136,팁이 더 가속화되는 것 아니냐는 우려가 커지고 있다,추론형,긍정,현재,확실,추론형-긍정-현재-확실


In [40]:
추론형_aug = copy.deepcopy(추론형)
추론형_aug['문장'] = 추론형_augtext

for idx in range(len(추론형_augtext)):
    if 추론형_augtext[idx][-2:] in end_word_2:
        continue
    elif 추론형_augtext[idx][-1] in end_word_1:
        continue
    invalid_index.append(idx)

추론형_aug = 추론형_aug.reset_index()
del 추론형_aug["index"]
추론형_aug = 추론형_aug.drop(index=invalid_index).reset_index()
del 추론형_aug['index']

추론형_words = set([
    "왜", "수 있", "수 없", "위해", "것 같", "될 것", "이 있", "있으므로", "이유", "때문", "같아", "같다", "것입니다", "것이다", 
    "미지수", "듯 하다", "듯하다", "의미다", "당연", "모양새", "만하다", "수밖에", "수 밖에", "해석", "분명", "셈", "하다는", "핵심", 
    "장담", "노릇", "해 볼", "터다", "야 한다", ])

invalid_index = []
for idx in range(len(추론형_aug)):
    text = 추론형_aug['문장'][idx]
    flag = False
    for word in 추론형_words:
        if word in text:
            flag = True
            break
    if flag == True:
        continue
    else:
        invalid_index.append(idx)

추론형_aug = 추론형_aug.drop(index=invalid_index).reset_index()
del 추론형_aug['index']
추론형_aug

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_16070,몸 전체에 퍼질 수 있으므로 주의해야 합니다,추론형,긍정,현재,불확실,추론형-긍정-현재-불확실
1,TRAIN_09703,생명을 잃은 사회로 전락하는 것입니다,추론형,긍정,미래,확실,추론형-긍정-미래-확실
2,TRAIN_08754,적의 은폐와 통신 블록이 위치하기 때문입니다,추론형,긍정,현재,확실,추론형-긍정-현재-확실
3,TRAIN_14369,상업적인 의도를 위한 것이라면 저는 이것을 하지 않았을 것입니다,추론형,부정,과거,불확실,추론형-부정-과거-불확실
4,TRAIN_03006,원산지 및 종축업자의 등급 정보를 쉽게 확인할 수 있습니다,추론형,긍정,미래,확실,추론형-긍정-미래-확실
...,...,...,...,...,...,...,...
206,TRAIN_07796,우리가 할 수 있는 일이 없다는 것을 알아야 합니다,추론형,긍정,미래,확실,추론형-긍정-미래-확실
207,TRAIN_14982,당신은 당신이 계약을 어기지 않았다고 주장할 수 있습니다,추론형,긍정,미래,불확실,추론형-긍정-미래-불확실
208,TRAIN_00048,원칙과 기준을 아는 것이 큰 도움이 될 것입니다,추론형,긍정,미래,확실,추론형-긍정-미래-확실
209,TRAIN_01233,산업과 지역의 경제를 활성화하는 데 사용할 수 있습니다,추론형,긍정,현재,확실,추론형-긍정-현재-확실


In [41]:
추론형_aug.to_csv("추론형_aug.csv", index=False)

In [ ]:
train = 